# Welcome to Mellea (Docs: Welcome, Quickstart)

Welcome to the interactive demo for **Mellea**, a library for writing generative programs. 
In this notebook, we will explore how Mellea allows you to write programs that generate content, validate it against requirements, and repair it if necessary.

**Docs Reference:** [Welcome](https://docs.mellea.ai/overview/mellea-welcome), [Quickstart](https://docs.mellea.ai/overview/quick-start)

In [3]:
import mellea
from mellea.backends import ModelOption
from mellea.backends.ollama import OllamaModelBackend

# Initialize Mellea with the local Ollama backend and Granite model
print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
        model_options={ModelOption.SEED: 42} # Fixed seed for reproducibility
    )
)
print("Session started! Model: granite4:micro")

Initializing Mellea Session...
Session started! Model: granite4:micro


## Your First Instruction

The core of Mellea is the `instruct` method. You give it a natural language instruction, and it returns a generated result.
Let's start with something simple.

In [4]:
response = m.instruct("Write a short haiku about writing python code.")
print(response)

  0%|          | 0/2 [00:00<?, ?it/s]

=== 16:34:05-INFO ======
SUCCESS


  0%|          | 0/2 [00:06<?, ?it/s]

Code in Python flows,
Elegance meets simplicity.
Logic takes its stage.


# The Loop: Instruct-Validate-Repair (Docs: Requirements)

Generative models can be unpredictable. Mellea's power lies in its **Instruct-Validate-Repair** loop. 
You can define **Requirements** (`req`) and **Checks** (`check`) that the output *must* satisfy. If the model fails, Mellea uses a **Strategy** to try again or fix it.

**Docs Reference:** [Requirements](https://docs.mellea.ai/overview/requirements)

### Example: Generating a Helper Email with Constraints
We want to generate an email that:
1.  Has a salutation.
2.  Does **not** use the word 'regards'.
3.  (Optional strict check) Uses only lower-case letters (just to show validation failure and repair).

In [5]:
from mellea.stdlib.requirements import req, check, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

# Define requirements
requirements = [
    req("The email should have a salutation"),
    check("The email should not mention the word 'regards'"),
    # req("Use only lower-case letters", validation_fn=simple_validate(lambda s: s.islower())) # Uncomment to test strict validation failure
]

def write_email(name, notes):
    print(f"Generating email for {name}...")
    email_candidate = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=requirements,
        # RejectionSamplingStrategy tries up to 'loop_budget' times to get a result that passes requirements
        strategy=RejectionSamplingStrategy(loop_budget=5),
        user_variables={"name": name, "notes": notes},
        return_sampling_results=True,
    )
    
    if email_candidate.success:
        print("\nSuccess! Email generated:")
        return str(email_candidate.result)
    else:
        print("\nFailed to meet requirements.")
        return email_candidate.sampling_results[0].value

# Test it
print(write_email("Arush", "Thanks for helping with the team project."))

Generating email for Arush...


  0%|          | 0/5 [00:00<?, ?it/s]

=== 16:34:11-INFO ======
SUCCESS


  0%|          | 0/5 [00:05<?, ?it/s]


Success! Email generated:
Subject: Heartfelt Thanks for Your Contribution

Dear Arush,

I hope this message finds you well. I am writing to express my sincere gratitude for your invaluable contribution towards our recent team project.

Your dedication, creativity and commitment did not go unnoticed, and they played an instrumental role in achieving the successful outcome of the project. The quality of work produced was truly impressive and a testament to your skills and abilities.

Please allow me to reiterate how much we appreciate your hard work and all that you have contributed. Your efforts have been crucial in making this project a success.

Thank you once again for everything.

Best regards,

[Your Name]


# Advanced: Core Concepts & Generative Programming

**Docs Reference:** [Generative Programming](https://docs.mellea.ai/overview/project-mellea), [Core Concepts](https://docs.mellea.ai/core-concept/generative-slots)

In this section, we touch on more advanced concepts. Mellea treats prompts not just as strings, but as programs with **Slots**, **Context**, and **Agents**.

### Variable Injection (Context)
You saw `user_variables` above. This is part of Mellea's Context Management system, allowing you to inject data safely into prompts.

In [6]:
# Quick example of dynamic variables
response = m.instruct(
    "Translate the following word to Spanish: {{word}}",
    user_variables={"word": "Hello"}
)
print(f"Hello in Spanish: {response}")

  0%|          | 0/2 [00:00<?, ?it/s]

=== 16:34:11-INFO ======
SUCCESS


  0%|          | 0/2 [00:00<?, ?it/s]

Hello in Spanish: The translation of "Hello" in Spanish is "Hola".


In [7]:
# Enter your instruction below
my_instruction = "Explain quantum computing to a 5 year old."

result = m.instruct(my_instruction)
print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

=== 16:34:18-INFO ======
SUCCESS


  0%|          | 0/2 [00:06<?, ?it/s]

Sure! Imagine you have a big box of toys. Now, when you want to play with your favorite toy car, in the regular games or world (which we can call 'classical computing'), it's like saying "Okay, let's only look for this car right now". You don't find other cars, you just search one by one until you find it.

Now, quantum computing is more like having a magic box of toys. In this magical box, your toy car and all the other cars can be in many places at once! It's kind of like your car being both here and there at the same time. This makes it much faster to find what you want because everything isn't stuck just in one place.

So, quantum computing is a bit different from our regular games or world - it's more magical and can do things really fast that we can't do otherwise!


## Generative Slots

``GenerativeSlot`` is a function whose implementation is provieded by an LLM.In Mellea you define these using ``@generative`` decorator


In [12]:
from typing import Literal
from mellea import generative, start_session


@generative
def classify_sentiment(text: str) -> Literal["positive", "negative", "neutral"]:
    """Classify the sentiment of the input text as 'positive', 'negative', or 'neutral'."""
    ...
 
m = start_session()
sentiment = classify_sentiment(m, text = "I love this product!")
print(sentiment)
    

=== 17:13:13-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=SimpleContext
positive


In [ ]:
# from mellea.std.base import ChatContext is old 
from mellea.stdlib.context import ChatContext
from mellea import start_session
from mellea.backends import ModelOption
from mellea import generative   

# ChatContext is used to maintain conversation history across multiple model calls. 
# Unlike SimpleContext (the default), which resets the chat history on each call, 
# ChatContext behaves like a chat history where previous messages are remembered.

@generative
def grade_syntax(code: str) -> int:
    """ Grade the code based on correct implementation of the function
    args:
        code: str (to be graded)
    returns:
        int : a grade between 1(worst) and 10(best)
    """
codes = (
    " def add(a, b):\n    return a + b",
    "def subtract(a,b): cout<<a-b<<endl",
    "int multiply(int a,int b) { return a*b}"
)
m = start_session(ctx = ChatContext() , model_options = {ModelOption.MAX_NEW_TOKENS: 10})
for code in codes :
    grade = grade_syntax(m, code = code)
    print("grade = ",grade)


=== 17:26:23-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=ChatContext, model_options={'@@@max_new_tokens@@@': 10}
grade =  5
